# Vision Transformer (ViT) 源码实现

本 Notebook 从零实现 Vision Transformer，包含所有核心模块和测试。

## 实现内容

### Part 1: 基础模块
- Patch Embedding 模块
- Position Embedding 模块
- 单元测试验证

### Part 2: 核心组件
- Multi-Head Self-Attention 机制
- Transformer Encoder Block
- 完整 ViT 模型
- 前向传播测试

## Part 1: 基础模块实现

### 1.1 导入依赖

In [1]:
import torch
import torch.nn as nn

print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

PyTorch 版本: 2.9.1+cu130
CUDA 可用: True


### 1.2 实现 Patch Embedding

Patch Embedding 将图像切分成固定大小的 patches，并通过卷积层映射到嵌入空间。

**核心思想**：
- 使用 kernel_size=patch_size, stride=patch_size 的卷积来切分图像
- 将 2D 特征图展平为序列

In [2]:
class PatchEmbed(nn.Module):
    """图像到Patch嵌入的转换"""
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        # 使用卷积实现patch切分和线性投影
        self.proj = nn.Conv2d(
            in_chans,
            embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )

    def forward(self, x):
        # x: (B, C, H, W)
        x = self.proj(x)            # (B, embed_dim, H/ps, W/ps)
        x = x.flatten(2)            # (B, embed_dim, num_patches)
        x = x.transpose(1, 2)       # (B, num_patches, embed_dim)
        return x

print("✅ PatchEmbed 实现完成")

✅ PatchEmbed 实现完成


### 1.3 测试 Patch Embedding

验证输出维度是否正确：
- 输入：(B, 3, 224, 224)
- 输出：(B, 196, 768) - 196 = (224/16)^2

In [3]:
# 测试 Patch Embedding
def test_patch_embed():
    x = torch.randn(2, 3, 224, 224)
    pe = PatchEmbed()
    out = pe(x)
    
    print(f"输入形状: {x.shape}")
    print(f"输出形状: {out.shape}")
    print(f"预期形状: (2, 196, 768)")
    
    assert out.shape == (2, 196, 768), f"维度错误: {out.shape}"
    print("✅ Patch Embedding 测试通过")
    return out

patch_output = test_patch_embed()

输入形状: torch.Size([2, 3, 224, 224])
输出形状: torch.Size([2, 196, 768])
预期形状: (2, 196, 768)
✅ Patch Embedding 测试通过


### 1.4 实现 Position Embedding

Position Embedding 为序列添加位置信息：
- CLS token: 用于分类的特殊token
- Position embeddings: 可学习的位置编码

In [4]:
class ViTEmbeddings(nn.Module):
    """ViT的嵌入层：添加CLS token和位置编码"""
    def __init__(self, num_patches, embed_dim):
        super().__init__()
        # CLS token: 用于分类的特殊token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        # 位置编码: num_patches + 1 (包含CLS token)
        self.pos_embed = nn.Parameter(
            torch.zeros(1, num_patches + 1, embed_dim)
        )

    def forward(self, x):
        # x: (B, N, D) - N是patch数量
        B = x.shape[0]
        
        # 扩展CLS token到batch
        cls_tokens = self.cls_token.expand(B, -1, -1)
        
        # 拼接CLS token到序列开头
        x = torch.cat((cls_tokens, x), dim=1)
        
        # 添加位置编码
        x = x + self.pos_embed
        return x

print("✅ ViTEmbeddings 实现完成")

✅ ViTEmbeddings 实现完成


### 1.5 测试 Position Embedding

验证 CLS token 和位置编码是否正确添加：
- 输入：(B, 196, 768)
- 输出：(B, 197, 768) - 多了1个CLS token

In [5]:
# 测试 Position Embedding
def test_pos_embed():
    x = torch.randn(2, 196, 768)
    emb = ViTEmbeddings(num_patches=196, embed_dim=768)
    out = emb(x)
    
    print(f"输入形状: {x.shape}")
    print(f"输出形状: {out.shape}")
    print(f"预期形状: (2, 197, 768) - 多了1个CLS token")
    
    assert out.shape == (2, 197, 768), f"维度错误: {out.shape}"
    print("✅ Position Embedding 测试通过")
    return out

pos_output = test_pos_embed()

输入形状: torch.Size([2, 196, 768])
输出形状: torch.Size([2, 197, 768])
预期形状: (2, 197, 768) - 多了1个CLS token
✅ Position Embedding 测试通过


### 1.6 Part 1 总结

In [6]:
print("="*50)
print("Part 1 完成！")
print("="*50)
print("✅ Patch Embedding 实现并测试通过")
print("✅ Position Embedding 实现并测试通过")
print("\n准备进入 Part 2: 实现注意力机制和Transformer...")
print("="*50)

Part 1 完成！
✅ Patch Embedding 实现并测试通过
✅ Position Embedding 实现并测试通过

准备进入 Part 2: 实现注意力机制和Transformer...


---

## Part 2: 核心组件实现

### 2.1 实现 Multi-Head Self-Attention

多头自注意力机制是 Transformer 的核心：
1. 线性投影生成 Q, K, V
2. 分割为多个头
3. 计算缩放点积注意力
4. 合并多头输出

In [7]:
class MultiHeadSelfAttention(nn.Module):
    """多头自注意力机制"""
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim 必须能被 num_heads 整除"
        
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # 一次性生成 Q, K, V
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        # 输出投影
        self.proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, N, D = x.shape

        # 1. 生成 Q, K, V
        qkv = self.qkv(x)  # (B, N, 3*D)
        qkv = qkv.reshape(B, N, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, B, heads, N, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # 2. 计算注意力分数（缩放点积）
        attn = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn = attn.softmax(dim=-1)  # (B, heads, N, N)

        # 3. 应用注意力到 V
        out = attn @ v  # (B, heads, N, head_dim)
        
        # 4. 合并多头
        out = out.transpose(1, 2).reshape(B, N, D)
        
        # 5. 输出投影
        out = self.proj(out)
        return out

print("✅ MultiHeadSelfAttention 实现完成")

✅ MultiHeadSelfAttention 实现完成


### 2.2 测试 Multi-Head Self-Attention

In [8]:
# 测试注意力机制
def test_attention():
    x = torch.randn(2, 197, 768)
    attn = MultiHeadSelfAttention(embed_dim=768, num_heads=12)
    out = attn(x)
    
    print(f"输入形状: {x.shape}")
    print(f"输出形状: {out.shape}")
    print(f"预期: 输入输出形状相同")
    
    assert out.shape == x.shape, f"维度错误: {out.shape}"
    print("✅ Multi-Head Self-Attention 测试通过")
    return out

attn_output = test_attention()

输入形状: torch.Size([2, 197, 768])
输出形状: torch.Size([2, 197, 768])
预期: 输入输出形状相同
✅ Multi-Head Self-Attention 测试通过


### 2.3 实现 Transformer Encoder Block

Encoder Block 结构：
1. LayerNorm + Multi-Head Attention + 残差连接
2. LayerNorm + MLP + 残差连接

使用 Pre-Norm 架构（更稳定）

In [9]:
class EncoderBlock(nn.Module):
    """Transformer Encoder Block"""
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0):
        super().__init__()
        
        # 第一个子层: 多头自注意力
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)

        # 第二个子层: MLP (Feed-Forward)
        self.norm2 = nn.LayerNorm(embed_dim)
        hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, embed_dim),
        )

    def forward(self, x):
        # Pre-Norm 架构
        # 子层1: 注意力 + 残差
        x = x + self.attn(self.norm1(x))
        # 子层2: MLP + 残差
        x = x + self.mlp(self.norm2(x))
        return x

print("✅ EncoderBlock 实现完成")

✅ EncoderBlock 实现完成


### 2.4 测试 Encoder Block

In [10]:
# 测试 Encoder Block
def test_encoder_block():
    x = torch.randn(2, 197, 768)
    block = EncoderBlock(embed_dim=768, num_heads=12)
    out = block(x)
    
    print(f"输入形状: {x.shape}")
    print(f"输出形状: {out.shape}")
    print(f"预期: 输入输出形状相同")
    
    assert out.shape == x.shape, f"维度错误: {out.shape}"
    print("✅ Encoder Block 测试通过")
    return out

block_output = test_encoder_block()

输入形状: torch.Size([2, 197, 768])
输出形状: torch.Size([2, 197, 768])
预期: 输入输出形状相同
✅ Encoder Block 测试通过


### 2.5 实现完整的 Vision Transformer

整合所有模块构建完整的 ViT：
1. Patch Embedding
2. Position Embedding (含 CLS token)
3. 多层 Transformer Encoder
4. 分类头

In [11]:
class VisionTransformer(nn.Module):
    """完整的 Vision Transformer 模型"""
    def __init__(
        self,
        img_size=224,
        patch_size=16,
        in_chans=3,
        embed_dim=768,
        depth=12,
        num_heads=12,
        num_classes=1000,
    ):
        super().__init__()

        # 1. Patch Embedding
        self.patch_embed = PatchEmbed(
            img_size, patch_size, in_chans, embed_dim
        )
        num_patches = self.patch_embed.num_patches
        
        # 2. Position Embedding (含 CLS token)
        self.embeddings = ViTEmbeddings(num_patches, embed_dim)

        # 3. Transformer Encoder (多层)
        self.blocks = nn.ModuleList([
            EncoderBlock(embed_dim, num_heads)
            for _ in range(depth)
        ])

        # 4. 分类头
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # (B, C, H, W) -> (B, num_patches, embed_dim)
        x = self.patch_embed(x)
        
        # 添加 CLS token 和位置编码
        x = self.embeddings(x)

        # 通过 Transformer Encoder
        for blk in self.blocks:
            x = blk(x)

        # 最终归一化
        x = self.norm(x)
        
        # 使用 CLS token 进行分类
        cls_token = x[:, 0]
        return self.head(cls_token)

print("✅ VisionTransformer 实现完成")

✅ VisionTransformer 实现完成


### 2.6 测试完整模型

测试标准 ViT-Base 配置：
- 输入：224x224 RGB 图像
- Patch size: 16x16
- Embed dim: 768
- Depth: 12 层
- Heads: 12
- Classes: 1000 (ImageNet)

In [12]:
# 测试完整 ViT 模型
print("="*50)
print("测试完整 Vision Transformer 模型")
print("="*50)

model = VisionTransformer()
x = torch.randn(2, 3, 224, 224)
y = model(x)

print(f"\n输入形状: {x.shape}")
print(f"输出形状: {y.shape}")
print(f"预期形状: (2, 1000)")

if y.shape == (2, 1000):
    print("\n✅ 完整模型测试通过！")
else:
    print(f"\n❌ 测试失败: 输出形状 {y.shape} 不匹配")

# 统计参数量
total_params = sum(p.numel() for p in model.parameters())
print(f"\n模型参数量: {total_params:,}")
print(f"模型大小: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")

测试完整 Vision Transformer 模型

输入形状: torch.Size([2, 3, 224, 224])
输出形状: torch.Size([2, 1000])
预期形状: (2, 1000)

✅ 完整模型测试通过！

模型参数量: 86,567,656
模型大小: 330.23 MB (FP32)

输入形状: torch.Size([2, 3, 224, 224])
输出形状: torch.Size([2, 1000])
预期形状: (2, 1000)

✅ 完整模型测试通过！

模型参数量: 86,567,656
模型大小: 330.23 MB (FP32)


### 2.7 测试不同配置的 ViT

In [13]:
# 测试 ViT-Small (用于 CIFAR-10)
print("\n" + "="*50)
print("测试 ViT-Small 配置 (CIFAR-10)")
print("="*50)

model_small = VisionTransformer(
    img_size=32,
    patch_size=4,
    embed_dim=256,
    depth=6,
    num_heads=8,
    num_classes=10
)

x_small = torch.randn(4, 3, 32, 32)
y_small = model_small(x_small)

print(f"输入形状: {x_small.shape}")
print(f"输出形状: {y_small.shape}")
print(f"预期形状: (4, 10)")

if y_small.shape == (4, 10):
    print("✅ ViT-Small 测试通过")

total_params_small = sum(p.numel() for p in model_small.parameters())
print(f"\n模型参数量: {total_params_small:,}")
print(f"模型大小: {total_params_small * 4 / 1024 / 1024:.2f} MB (FP32)")


测试 ViT-Small 配置 (CIFAR-10)
输入形状: torch.Size([4, 3, 32, 32])
输出形状: torch.Size([4, 10])
预期形状: (4, 10)
✅ ViT-Small 测试通过

模型参数量: 4,771,082
模型大小: 18.20 MB (FP32)


### 2.8 可视化模型结构

In [14]:
# 打印模型结构摘要
print("\n" + "="*50)
print("ViT-Base 模型结构摘要")
print("="*50)
print(f"\n配置:")
print(f"  - 图像尺寸: 224x224")
print(f"  - Patch 尺寸: 16x16")
print(f"  - Patch 数量: {(224//16)**2}")
print(f"  - 嵌入维度: 768")
print(f"  - Transformer 层数: 12")
print(f"  - 注意力头数: 12")
print(f"  - MLP 隐藏层维度: {768*4}")
print(f"  - 类别数: 1000")

print(f"\n主要组件:")
print(f"  1. Patch Embedding (Conv2d)")
print(f"  2. CLS Token + Position Embedding")
print(f"  3. 12x Transformer Encoder Block")
print(f"     - Multi-Head Self-Attention (12 heads)")
print(f"     - MLP (768 -> 3072 -> 768)")
print(f"  4. Classification Head (Linear)")

print(f"\n总参数量: {total_params:,}")


ViT-Base 模型结构摘要

配置:
  - 图像尺寸: 224x224
  - Patch 尺寸: 16x16
  - Patch 数量: 196
  - 嵌入维度: 768
  - Transformer 层数: 12
  - 注意力头数: 12
  - MLP 隐藏层维度: 3072
  - 类别数: 1000

主要组件:
  1. Patch Embedding (Conv2d)
  2. CLS Token + Position Embedding
  3. 12x Transformer Encoder Block
     - Multi-Head Self-Attention (12 heads)
     - MLP (768 -> 3072 -> 768)
  4. Classification Head (Linear)

总参数量: 86,567,656


### 2.9 Part 2 总结

In [15]:
print("\n" + "="*50)
print("Part 2 完成！")
print("="*50)
print("✅ Multi-Head Self-Attention 实现并测试通过")
print("✅ Transformer Encoder Block 实现并测试通过")
print("✅ 完整 ViT 模型实现并测试通过")
print("✅ 模型可正常前向传播")
print("="*50)


Part 2 完成！
✅ Multi-Head Self-Attention 实现并测试通过
✅ Transformer Encoder Block 实现并测试通过
✅ 完整 ViT 模型实现并测试通过
✅ 模型可正常前向传播


---

## 最终总结

In [16]:
print("\n" + "="*60)
print(" "*15 + "Vision Transformer 实现完成！")
print("="*60)
print("\n✅ Part 1: 基础模块")
print("   - Patch Embedding")
print("   - Position Embedding")
print("\n✅ Part 2: 核心组件")
print("   - Multi-Head Self-Attention")
print("   - Transformer Encoder Block")
print("   - 完整 ViT 模型")
print("\n✅ 所有单元测试通过")
print("✅ 模型可正常前向传播")
print("\n📝 实现的模型配置:")
print(f"   - ViT-Base: {total_params:,} 参数")
print(f"   - ViT-Small: {total_params_small:,} 参数")
print("\n🎉 现在可以使用这个模型进行训练了！")
print("="*60)


               Vision Transformer 实现完成！

✅ Part 1: 基础模块
   - Patch Embedding
   - Position Embedding

✅ Part 2: 核心组件
   - Multi-Head Self-Attention
   - Transformer Encoder Block
   - 完整 ViT 模型

✅ 所有单元测试通过
✅ 模型可正常前向传播

📝 实现的模型配置:
   - ViT-Base: 86,567,656 参数
   - ViT-Small: 4,771,082 参数

🎉 现在可以使用这个模型进行训练了！
